In [1]:
print("test")

test


In [2]:
import math
from tqdm import tqdm
import scvi
import torch
import numpy as np
import pytorch_lightning as pl
import os

import scanpy as sc

from pytorch_lightning.callbacks import TQDMProgressBar
from torchcfm.conditional_flow_matching import *
import scanpy as sc
from datamodules import  SCFMDataset, cfm_collate, StratifiedBatchSampler, ot_collate
from torch.utils.data import RandomSampler
from sc_etl_utils import *
from arch import *
from flow_utils import compute_conditional_flow
import json

import scvi
import torch
import numpy as np
import pytorch_lightning as pl
import os

import scanpy as sc

from torchcfm.conditional_flow_matching import *
import scanpy as sc
import hashlib
from llm import MAE

In [3]:
# load some data
adata = sc.read_h5ad('/orcd/archive/abugoot/001/Projects/dlesman/datasets/satija_IFNB_HVG_and_perturbed_genes_raw.h5ad')

In [4]:
gene_map = {k: i for i, k in enumerate(adata.var.index)}
gene_map = gene_map | {'NT': max(gene_map.values()) + 1}
gene_unmap = {gene_map[k]: k for k in gene_map}
perts = adata.obs.gene.unique().map(gene_map)
adata.obs['pert_type'] = adata.obs.gene.map(gene_map)
pert_ids = np.array(adata.obs['pert_type'])
pert_mat = np.arange(pert_ids.max() + 1)[:, None]

In [5]:
cell_col = 'cell_type'
pert_col = 'pert_type'
control_pert, holdout_cells, holdout_perts = gene_map['NT'], ['HT29'], [gene_map['USP18']]

In [6]:
from sc_etl_utils import *
control_idx, pert_idx, eval_idx, eval_cell_idx, eval_pert_idx = get_train_eval_idxs(
    adata, control_pert, holdout_cells, holdout_perts, cell_col=cell_col, pert_col=pert_col
)

_, _, cell_types = get_identity_features(
    adata, cell_col=cell_col, pert_col=pert_col, cell_type_features=False
)

adata.obsm["standard"] = adata.X
X = adata.obsm["standard"]
X = X.toarray()
X = X / X.sum(axis=1)[:, None]

control_train, pert_train, pert_ids_train, control_cell_types, pert_cell_types, control_eval, pert_eval, pert_ids_eval = get_train_eval(
    X, pert_ids, cell_types, control_idx, pert_idx, eval_idx, eval_cell_idx, eval_pert_idx
)


Controls: 14582, Perturbations: 313080,  Eval: 880


In [7]:
from OTmap import *

In [8]:
import math
from tqdm import tqdm
import torch
import numpy as np
import os
from typing import Optional, Union, Iterator, Iterable, Optional, Sequence, List, TypeVar, Generic, Sized, Union
import scanpy as sc
from torch.utils.data import RandomSampler
import json
import sys
import hashlib
from sc_etl_utils import *
from functools import partial
import ot as pot
from torch.utils.data import BatchSampler, SequentialSampler, Sampler
from torch import nn
import anndata as ad
import scipy
from scipy.sparse import issparse
import anndata
import pandas as pd  
import pickle
from matplotlib import pyplot as plt
from scipy.optimize import linear_sum_assignment
from torch.utils.data import Dataset, DataLoader
import random
import scipy.stats as stats


def compute_ot_mapping(adata, cost_threshold=0.01):
    mappings = {}
    valid_perturbed_indices = []

    for cell_type in adata.obs['cell_type'].unique():
        for gene in adata.obs['gene'].unique():
            if gene == 'NT':
                continue
            
            # Filter data for the specific cell type and gene
            perturbed_cells = adata[(adata.obs['cell_type'] == cell_type) & (adata.obs['gene'] == gene)]
            control_cells = adata[(adata.obs['cell_type'] == cell_type) & (adata.obs['gene'] == 'NT')]
            
            # Extract total counts for both sets of cells
            perturbed_counts = perturbed_cells.obs['nCount_RNA'].values
            control_counts = control_cells.obs['nCount_RNA'].values
            
            # Compute the cost matrix based on the absolute difference in total counts
            cost_matrix = np.abs(perturbed_counts[:, None] - control_counts[None, :])
            
            # Solve the linear sum assignment problem
            row_ind, col_ind = linear_sum_assignment(cost_matrix)
            
            # Discard mappings that differ by more than 1 percent in their counts (or whatever you set cost_threshold to)
            valid_mapping = cost_matrix[row_ind, col_ind] / perturbed_counts[row_ind] <= cost_threshold
            
            # Save the valid perturbed indices
            valid_perturbed_indices.extend(perturbed_cells.obs.index[row_ind[valid_mapping]].tolist())
            
            # Save the mappings
            mappings[(cell_type, gene)] = {
                "perturbed_indices": perturbed_cells.obs.index[row_ind[valid_mapping]].tolist(),
                "control_indices": control_cells.obs.index[col_ind[valid_mapping]].tolist()
            }
    
    return mappings, valid_perturbed_indices

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
datasetso = 'satija_IFN'

inp = adata # sc.read(f'input/{datasetso}.h5ad')

#inp = inp[inp.obs['cell_type']=='A549']

inp.var_names = inp.var['gene']

hold = 'IFNAR2'

inp = inp[inp.obs['gene']!=hold]


# Convert sparse matrix to dense
inp.X = np.array(inp.X.todense())

# Convert the relevant obs and var columns to numpy arrays
inp.obs['nCount_RNA'] = inp.obs['nCount_RNA'].values
inp.obs['gene'] = inp.obs['gene'].values
inp.obs['cell_type'] = inp.obs['cell_type'].values
inp.var['gene'] = inp.var['gene'].values

ot_mappings, valid_perturbed_indices = compute_ot_mapping(inp, cost_threshold=0.01)

#exdample usage of ot_mapping
print(f"First five control indices for cell_type=BXPC3, perturbation=IFNAR1 are: {ot_mappings[('BXPC3','IFNAR1')]['perturbed_indices'][:5]}")
print(f"First five pert indices for cell_type=BXPC3, perturbation=IFNAR1 are: {ot_mappings[('BXPC3','IFNAR1')]['control_indices'][:5]}")

#example usage of dataloader: 

#for control, pert, pert_index, pert_expr in (pbar := tqdm(dataloader, desc="Loading Batches")): 
    #... etc

/tmp/ipykernel_3253088/521056526.py:19: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  inp.obs['nCount_RNA'] = inp.obs['nCount_RNA'].values


First five control indices for cell_type=BXPC3, perturbation=IFNAR1 are: ['24_41_31_1_1_1_1_1_1_1_1_1', '23_15_51_1_1_1_1_1_1_1_1_1', '22_46_83_1_1_1_1_1_1_1_1_1', '24_61_01_1_1_1_1_1_1_1_1_1', '24_81_15_1_1_1_1_1_1_1_1_1']
First five pert indices for cell_type=BXPC3, perturbation=IFNAR1 are: ['24_05_46_2_1_1_1_1_2', '23_49_88_2_1_1', '24_08_85_1_1_1_1_1_1_1_1_2', '24_20_29_2_1_1_1_1_2', '21_92_57_2_1_1_1']


In [10]:
inp.X = inp.X.toarray()
inp.X = inp.X / inp.X.sum(axis=1)[:, None]

In [11]:
class OTMappingDataset(Dataset):
    def __init__(self, adata, mappings, valpertind):
        self.adata = adata
        self.mappings = mappings
        self.valpertind = valpertind
        self.gene_map = {
            gene: np.where(self.adata.var['gene'] == gene)[0][0] for gene in self.adata.var['gene']
        }

    def __len__(self):
        return len(self.valpertind)

    def __getitem__(self, idx, mixup=False):
        # print(0)
        
        curpert = self.adata[self.valpertind[idx]]

        gene = curpert.obs['gene'].item()
        cell_type = curpert.obs['cell_type'].item()

        
        pert_mapping = self.mappings[(cell_type, gene)]
        ridex = pert_mapping['perturbed_indices'].index(curpert.obs_names[0])
        # print(pert_mapping['perturbed_indices'].index)
        perturbed_index = pert_mapping['perturbed_indices'][ridex]
        control_index = pert_mapping['control_indices'][ridex]
        
        # idx = np.random.choice(minibatch_size, minibatch_size, replace=False)
        # control = (1 - t) * control + t * control[idx]
        # pert = (1 - t) * control + t * control[idx]
        

        # Get the perturbed cell and the corresponding NT cell
        perturbed_cell = self.adata[perturbed_index].X
        nt_cell = self.adata[control_index].X
        
        # Convert everything to torch tensors and return
        # nt_cell_tensor = torch.tensor(nt_cell.X, dtype=torch.float32)# .cuda()
        # perturbed_cell_tensor = torch.tensor(perturbed_cell.X, dtype=torch.float32)# .cuda()
        # gene_index_tensor = torch.tensor(gene_index, dtype=torch.long)# .cuda()
        # expression_value_tensor = torch.tensor(expression_value, dtype=torch.float32)# .cuda()
        
        if mixup:
            t = np.random.beta(1, 1)
            r2 = np.random.choice(len(pert_mapping['perturbed_indices']))
            perturbed_index2 = pert_mapping['perturbed_indices'][r2]
            control_index2 = pert_mapping['control_indices'][r2]
            perturbed_cell2 = self.adata[perturbed_index2].X
            nt_cell2 = self.adata[control_index2].X
            perturbed_cell = t * perturbed_cell + (1 - t) * perturbed_cell2
            nt_cell = t * nt_cell + (1 - t) * nt_cell2
        
        # Find the index of the perturbed gene in var['gene']
        # gene_index = np.where(self.adata.var['gene'] == gene)[0][0]
        gene_index = self.gene_map[gene]
        
        # Get the expression value for the perturbed cell at that gene
        # expression_value = perturbed_cell.X[0, gene_index]

        # print(2)

        return nt_cell, perturbed_cell, gene_index

dataset = OTMappingDataset(inp, ot_mappings, valid_perturbed_indices)
dl = DataLoader(dataset, batch_size=32, shuffle=True)

In [12]:
dl = dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [29]:
def ot_collate(
    batch,
    return_noise=False,
    ot_sampler = OTPlanSampler(method="exact")
):

    batch = list(zip(*batch))
    batch = [torch.tensor(np.array(x)) for x in batch]
    control, target = batch[:2]
    
    x0, x1 = ot_sampler.sample_plan(10000 * control, 10000 * target)

    return x0 / 10000, x1 / 10000, *batch[2:]

In [30]:
batch_size = 512
dset = SCFMDataset(
    control_train, pert_train, 
    pert_ids_train, pert_mat, 
    control_cell_types, pert_cell_types,
    batch_size=batch_size, size=X.shape[0]
)
ns = np.array([[t.shape[0] for t in ts] for ts in dset.target])
dl_ = torch.utils.data.DataLoader(
    dset, collate_fn=ot_collate, 
    batch_sampler=StratifiedBatchSampler(
        ns=ns, batch_size=512
    )
)

Strata probs [0.         0.0001086  0.00030344 0.00039606 0.00040565 0.00041523
 0.00042162 0.00043439 0.00045356 0.00045995 0.00049189 0.00052063
 0.00054938 0.00056535 0.00061007 0.00062284 0.00062604 0.00062604
 0.00063562 0.00068673 0.00075061 0.00076977 0.00077297 0.00077935
 0.00079852 0.0008081  0.00084323 0.0008624  0.00087837 0.00088156
 0.00088795 0.00088795 0.00089753 0.00089753 0.0009167  0.00092628
 0.00093906 0.00094864 0.00099016 0.00099016 0.00100294 0.00104127
 0.00104446 0.00105085 0.00109237 0.00110834 0.00110834 0.00111154
 0.00111473 0.00111473 0.0011307  0.0011339  0.00113709 0.00116584
 0.00117222 0.00117222 0.00121375 0.00121694 0.00122333 0.00122652
 0.00124569 0.00124888 0.00124888 0.00126485 0.00128082 0.00129999
 0.00130318 0.00130318 0.00131596 0.00131596 0.00132554 0.00133193
 0.00133512 0.0013447  0.00135109 0.00137026 0.00137984 0.00137984
 0.00139262 0.0014022  0.00140539 0.00140539 0.00141178 0.00141497
 0.00141817 0.00143733 0.00146288 0.00146608 0.00

In [13]:
from llm import *


class MAE_Decoder(torch.nn.Module):
    def __init__(self,
                 pos_embedding,
                 num_layer=6,
                 num_head=3,
                 ff_dim=128,
                 true_sparsity=True,
                 expr_activation='sigmoid'
                 ) -> None:
        super().__init__()
        
        _, input_dim, emb_dim = pos_embedding.pos.shape
        self.pos_embedding = pos_embedding

        self.mask_token = torch.nn.Parameter(torch.zeros(1, emb_dim))

        self.transformer = torch.nn.Sequential(
            *[
                nn.TransformerEncoderLayer(
                    emb_dim, num_head, dim_feedforward=ff_dim, batch_first=True, dropout=0.
                ) 
                for _ in range(num_layer)
            ]
        )

        self.head = torch.nn.Linear(emb_dim, 2)
        self.sigmoid = torch.nn.Sigmoid()
        self.sparsity = BernoulliSampleLayer()
        self.true_sparsity = true_sparsity
        if expr_activation == 'sigmoid':
            self.expr_activation = torch.nn.Sigmoid()
        elif expr_activation == 'identity':
            self.expr_activation = lambda x: x
        else:
            print("Error: expr_activation must be sigmoid or identity")
        
        nn.init.normal_(self.mask_token, std=.02)


    def forward(self, features, backward_indexes, pert_features=None):
        T = features.shape[1]
        features = torch.cat(
            [features, self.mask_token.expand(features.shape[0], backward_indexes.shape[1] - features.shape[1], -1)], dim=1
        )
        # we have to add this or the masked objective will not work!
        features = take_indexes(features, backward_indexes)
        features = features + self.pos_embedding.pos
        features_ = features
        
        if pert_features is not None:
            _, N, _ = pert_features.shape
            features = torch.cat([features, pert_features], dim=1)

        features = self.transformer(features)
        
        if pert_features is not None:
            features = features[:, :-N, :]
        expr_logits, sparsity_logits = self.head(features).unbind(-1)
        
        mask = torch.zeros_like(expr_logits)
        mask[:, T:] = 1
        mask = take_indexes(mask, backward_indexes)
        
        expr = self.sigmoid(expr_logits)
        
        if self.true_sparsity:
            sparsity_probs = self.sigmoid(sparsity_logits)
            sparsity = self.sparsity(sparsity_probs)
            expr = expr * sparsity
            return expr, sparsity_probs, mask

        return expr, mask


class MAE(torch.nn.Module):
    def __init__(self,
                 input_dim,
                 ff_dim=128,
                 emb_dim=48,
                 encoder_layer=6,
                 encoder_head=4,
                 decoder_layer=6,
                 decoder_head=4,
                 mask_ratio=0.75,
                 true_sparsity=True,
                 expr_activation='sigmoid'
                 ) -> None:
        super().__init__()
        
        self.pos_embedding = PosEmbedding(input_dim=input_dim, emb_dim=emb_dim)
        self.encoder = MAE_Encoder(
            self.pos_embedding, encoder_layer, encoder_head, mask_ratio, ff_dim=ff_dim)
        self.pert_embedding = PertEmbedder(self.encoder)
        self.decoder = MAE_Decoder(
            self.pos_embedding, decoder_layer, decoder_head, ff_dim=ff_dim,
            true_sparsity=true_sparsity, expr_activation=expr_activation
        )
        self.pert_decoder = MAE_Decoder(self.pos_embedding, decoder_layer, decoder_head, ff_dim=ff_dim,true_sparsity=true_sparsity, expr_activation=expr_activation)
        # self.pert_decoder = self.decoder

    def forward(self, batch, mask=True, pert_index=None, pert_expr=None, recon_and_pert=False):
        features, backward_indexes = self.encoder(batch, mask=mask)
        if pert_index is not None and pert_expr is not None:
            pert_features = self.pert_embedding(pert_index, pert_expr)
            if recon_and_pert:
                recon_out = self.decoder(features, backward_indexes)
                pert_out = self.pert_decoder(
                    features, backward_indexes, pert_features=pert_features
                )
                return recon_out, pert_out
            else:
                return self.pert_decoder(
                    features, backward_indexes, pert_features=pert_features
                )
        else:
            return self.decoder(features, backward_indexes)

In [62]:
model = MAE(
    X.shape[1], emb_dim=96, decoder_layer=4, encoder_layer=4, encoder_head=3, decoder_head=3, ff_dim=128,
    true_sparsity=True, expr_activation="sigmoid"
)
model = torch.load(f"llm/v6")
device = 'cuda'
# device = 'cpu'
model = model.to(device)

In [63]:
import copy
model.pert_decoder = copy.deepcopy(model.decoder)

In [90]:
base_learning_rate = 5e-5
weight_decay=0.0
total_epoch = 500
warmup_epoch = 10
minibatch_size = 128
save_dir = "llm/v"

Below are two training loops, first is just the autoencoder and second is the autoencoder and pert task. I usually run the autoencoder for a couple iterations then the joint task.

In [91]:
base_learning_rate = 5e-5
optim = torch.optim.AdamW(model.parameters(), lr=base_learning_rate, betas=(0.9, 0.999), weight_decay=weight_decay)
lr_func = lambda epoch: min((epoch + 1) / (warmup_epoch + 1e-5), 0.5 * (math.cos(epoch / total_epoch * math.pi) + 1))
lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optim, lr_lambda=lr_func, verbose=True)

Adjusting learning rate of group 0 to 5.0000e-06.


In [96]:
state_dict

OrderedDict([('pos_embedding.pos',
              tensor([[[ 0.1280,  0.2470, -1.1113,  ...,  0.1775,  0.7051, -0.1277],
                       [ 2.7049, -0.4807,  0.3927,  ...,  1.1225,  0.0638, -0.5066],
                       [ 1.2915, -0.3232,  0.7227,  ..., -1.1896,  0.5530, -0.7407],
                       ...,
                       [ 1.2119, -0.3661, -0.2215,  ...,  0.5652,  0.3100, -0.1641],
                       [ 0.5269, -0.9177, -0.2874,  ..., -1.9569, -1.0188,  0.1251],
                       [-0.2233, -0.7706,  0.6919,  ...,  0.0769,  0.5840, -0.3722]]],
                     device='cuda:0')),
             ('encoder.pos_embedding.pos',
              tensor([[[ 0.1280,  0.2470, -1.1113,  ...,  0.1775,  0.7051, -0.1277],
                       [ 2.7049, -0.4807,  0.3927,  ...,  1.1225,  0.0638, -0.5066],
                       [ 1.2915, -0.3232,  0.7227,  ..., -1.1896,  0.5530, -0.7407],
                       ...,
                       [ 1.2119, -0.3661, -0.2215,  ...,  0

In [97]:
state_dict = model.state_dict()
z = state_dict['decoder.head.weight']
z[1] = torch.zeros_like(z[1])
state_dict['decoder.head.weight'] = z
state_dict['pert_decoder.head.weight'] = z
model.load_state_dict(state_dict)

<All keys matched successfully>

In [98]:
use_sparsity_loss = True
use_mask_task = False
use_active_weights = True
lr_step = 32
minibatch_size = 32

In [ ]:
use_sparsity_loss = True
use_mask_task = False
use_active_weights = True
lr_step = 32
minibatch_size = 32

step_count = 0
optim.zero_grad()
pert_task = 0
for e in range(20):
    model.train()
    losses = {'control': [], 'pert': []}
    for (bcontrol, bpert, bpert_index) in (pbar := tqdm(iter(dl))):
        bcontrol, bpert, bpert_index = bcontrol.squeeze(), bpert.squeeze(), bpert_index.reshape(-1, 1)# , # bpert_expr.squeeze()
        curr_batch_size = bcontrol.shape[0]
        for i in range(curr_batch_size // minibatch_size):
            control = bcontrol[(i * minibatch_size):((i + 1) * minibatch_size)]
            if control.shape[0] == 0:
                continue
            pert = bpert[(i * minibatch_size):((i + 1) * minibatch_size)]
            pert_index = bpert_index[(i * minibatch_size):((i + 1) * minibatch_size)]
            pert_index = pert_index.squeeze()
            # print(pert_index)
            active_weights = 100 * (control > 0).float().to(device) + 1 if use_active_weights else 1
            pert_active_weights = 100 * (pert > 0).float().to(device) + 1 if use_active_weights else 1
            control = control.float().to(device)
            pert = pert.float().to(device)
            step_count += 1

            pert_expr = pert[torch.arange(pert.size(0)), pert_index, None]
            control_results, pert_results = model(
                control, pert_expr=pert_expr, pert_index=pert_index, mask=use_mask_task, recon_and_pert=True
            )
            
            control_recon, pert_recon = control_results[0], pert_results[0]
            control_loss = torch.sum(active_weights * torch.abs(control_recon - control)) / minibatch_size
            pert_loss = torch.sum(pert_active_weights * torch.abs(pert_recon - pert)) / minibatch_size
            
            mean_pert, mean_pert_recon = pert.mean(axis=0), pert_recon.mean(axis=0)
            mean_pert_loss = 10 * torch.sum(torch.abs(mean_pert - mean_pert_recon)) 
            
            std_pert, std_pert_recon = pert.std(axis=0), pert_recon.std(axis=0)
            std_pert_loss = 10 * torch.sum(torch.abs(std_pert - std_pert_recon)) 
            
            if use_sparsity_loss and len(control_results) == 3:
                control_sparsity, pert_sparsity = control_results[1], pert_results[1]
                control_loss += torch.sum(active_weights * torch.abs(control_sparsity - (control > 0).float())) / minibatch_size
                pert_loss += torch.sum(pert_active_weights * torch.abs(pert_sparsity - (pert > 0).float())) / minibatch_size

            loss = (pert_loss + control_loss + mean_pert_loss + std_pert_loss)
            loss.backward()
            optim.step()
            optim.zero_grad()
            losses['control'].append(control_loss.item())
            losses['pert'].append(pert_loss.item())
            if step_count % lr_step == 0:
                lr_scheduler.step()
            pbar.set_description(
                f"loss: {loss:.3f}, tv: {np.array(losses['control'])[-lr_step:].mean():.3f}, ptv: {np.array(losses['pert'])[-lr_step:].mean():.3f}"
            )
    
    avg_loss = sum(losses['control']) / len(losses['control'])
    torch.save(model, f"{save_dir}{e}")
    # writer.add_scalar('mae_loss', avg_loss, global_step=e)
    print(f'In epoch {e}, average traning loss is {avg_loss}.')

loss: 22705.689, tv: 11060.818, ptv: 10993.777:   0%|▎                                                                         | 32/9380 [00:32<2:39:38,  1.02s/it]

Adjusting learning rate of group 0 to 1.0000e-05.


loss: 20476.381, tv: 10967.484, ptv: 10949.576:   1%|▍                                                                        | 64/9380 [01:38<20:43:20,  8.01s/it]

Adjusting learning rate of group 0 to 1.5000e-05.


loss: 23596.961, tv: 10605.375, ptv: 10451.895:   1%|▊                                                                         | 96/9380 [02:11<2:39:15,  1.03s/it]

...

loss: 1839.710, tv: 538.747, ptv: 1230.729:  59%|███████████████████████████████████████████▎                              | 5498/9380 [1:38:50<1:07:32,  1.04s/it]

Adjusting learning rate of group 0 to 2.6256e-05.


loss: 1727.972, tv: 542.579, ptv: 1241.109:  59%|███████████████████████████████████████████▋                              | 5530/9380 [1:39:23<1:06:50,  1.04s/it]

Adjusting learning rate of group 0 to 2.6413e-05.


loss: 1752.865, tv: 538.179, ptv: 1221.573:  59%|███████████████████████████████████████████▊                              | 5550/9380 [1:39:44<1:09:27,  1.09s/it]

In [ ]:
control_sparsity, control

In [60]:
use_sparsity_loss = False
use_mask_task = False
use_active_weights = True
lr_step = 32

step_count = 0
optim.zero_grad()
pert_task = 0
for e in range(2):
    model.train()
    losses = {'control': [], 'pert': []}
    for (bcontrol, bpert, bpert_index, _) in (pbar := tqdm(iter(dl))):
        bcontrol, bpert, bpert_index = bcontrol.squeeze(), bpert.squeeze(), bpert_index.reshape(-1, 1)# , # bpert_expr.squeeze()
        for i in range(curr_batch_size // minibatch_size):
            control = bcontrol[(i * minibatch_size):((i + 1) * minibatch_size)]
            if control.shape[0] == 0:
                continue
            pert = bpert[(i * minibatch_size):((i + 1) * minibatch_size)]
            pert_index = bpert_index[(i * minibatch_size):((i + 1) * minibatch_size)]
            pert_index = pert_index.squeeze()
            # print(pert_index)
            active_weights = 1000 * (control > 0).float().to(device) + 1 if use_active_weights else 1
            pert_active_weights = 1000 * (pert > 0).float().to(device) + 1 if use_active_weights else 1
            control = control.float().to(device)
            pert = pert.float().to(device)
            step_count += 1

            pert_expr = pert[torch.arange(pert.size(0)), pert_index, None]
            control_results, pert_results = model(
                control, pert_expr=pert_expr, pert_index=pert_index, mask=use_mask_task, recon_and_pert=True
            )
            
            control_recon, pert_recon = control_results[0], pert_results[0]
            control_loss = torch.sum(active_weights * torch.abs(control_recon - control)) / minibatch_size
            pert_loss = torch.sum(pert_active_weights * torch.abs(pert_recon - pert)) / minibatch_size
            
            if use_sparsity_loss and len(control_results) == 3:
                control_sparsity, pert_sparsity = control_results[1], pert_results[1]
                control_loss += torch.sum(active_weights * torch.abs(control_sparsity - (control > 0).float())) / minibatch_size
                pert_loss += torch.sum(pert_active_weights * torch.abs(pert_sparsity - (pert > 0).float())) / minibatch_size

            loss = (pert_loss + control_loss)
            loss.backward()
            optim.step()
            optim.zero_grad()
            losses['control'].append(control_loss.item())
            losses['pert'].append(pert_loss.item())
            if step_count % lr_step == 0:
                lr_scheduler.step()
            pbar.set_description(
                f"tv: {np.array(losses['control'])[-lr_step:].mean():.3f}, ptv: {np.array(losses['pert'])[-lr_step:].mean():.3f}"
            )
    
    avg_loss = sum(losses['control']) / len(losses['control'])
    torch.save(model, f"{save_dir}{e}")
    # writer.add_scalar('mae_loss', avg_loss, global_step=e)
    print(f'In epoch {e}, average traning loss is {avg_loss}.')

  0%|                                                                                                                                     | 0/9380 [00:00<?, ?it/s]


ValueError: not enough values to unpack (expected 4, got 3)

In [45]:
cell_type_names = adata.obs[cell_col]
pert_type_names = adata.obs[pert_col]
# Save the predicted perturbation
for cell_type, pert_type in zip(holdout_cells, holdout_perts):
    control_eval =  torch.tensor(X[cell_type_names == cell_type]).float()# .to(device)
    true_pert= torch.tensor(X[(pert_type_names == pert_type) & (cell_type_names == cell_type)]).float()# .to(device)
    curr_batch_size = min(control_eval.shape[0], true_pert.shape[0])
    pred_perts = []
    for i in range(curr_batch_size // minibatch_size + 1):
        print(i)
        control = control_eval[(i * minibatch_size):min(curr_batch_size, ((i + 1) * minibatch_size))].to(device)
        pert = true_pert[(i * minibatch_size):min(curr_batch_size, ((i + 1) * minibatch_size))].to(device)
        pert_expr = pert[torch.arange(pert.shape[0]), pert_type, None]
        pred_pert, _, = model(
            control, pert_expr=pert_expr, pert_index=pert_type, mask=False
        )
        pred_pert = control.cpu()
        control.cpu().numpy()
        
        pert.cpu().numpy()
        pred_perts.append(pred_pert.cpu().detach().numpy())
        torch.cuda.empty_cache()
    pred_pert = np.vstack(pred_perts)
    print(f"Saving {gene_unmap[pert_type]} predictions")
    np.savez(
        f"Satija_IFNB_HVG/llm/pred_{gene_unmap[pert_type]}_{cell_type}.npz", 
        pred_pert=pred_pert, 
        true_pert=true_pert, 
        control=control_eval,
    )

0
1
2
3
4
5
6
Saving USP18 predictions


In [46]:
pert.std(axis=0).sort()

torch.return_types.sort(
values=tensor([0.0000, 0.0000, 0.0000,  ..., 0.0125, 0.0138, 0.0194], device='cuda:0'),
indices=tensor([   1,    2,    4,  ..., 2026, 1550, 1612], device='cuda:0'))